# AdventureWorks - Predicting customer churn

In this notebook, we will demonstrate how getML can be used for a customer churn project using a synthetic dataset of a fictional company. We will also benchmark getML against featuretools.

Summary:

- Prediction type: __Classification model__
- Domain: __Customer loyalty__
- Prediction target: __churn__ 
- Population size: __19704__

## Background

[AdventureWorks](https://github.com/Microsoft/sql-server-samples/releases/tag/adventureworks) is a fictional company that sells bicycles. It is used by Microsoft to showcase how its MS SQL Server can be used to manage business data. Since this dataset resembles a real-world customer database and it is open-source, we will use it to showcase how getML can be used for a classic customer churn project (real customer databases are not easily available for the purposes of showcasing and benchmarking, for reasons of data privacy).

The dataset has been downloaded from the [CTU Prague relational learning repository](https://relational.fit.cvut.cz/dataset/AdventureWorks) (Motl and Schulte, 2015) (Now residing at [relational-data.org](https://relational-data.org/dataset/AdventureWorks).).

We will benchmark [getML](https://www.getml.com) 's feature learning algorithms against [featuretools](https://www.featuretools.com), an open-source implementation of the propositionalization algorithm, similar to getML's FastProp.

## Analysis

Let's get started with the analysis and set up your session:

In [61]:
%pip install -q "getml==1.5.0" "featuretools==1.31.0" "ipywidgets==8.1.5"

In [2]:
import os
import warnings
import numpy as np
import pandas as pd

import featuretools
import woodwork as ww
import getml

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
warnings.simplefilter(action='ignore', category=FutureWarning)

In [60]:
getml.engine.launch(allow_remote_ips=True, token='token')
getml.set_project('adventure_works')

Launching ./getML --allow-push-notifications=true --allow-remote-ips=true --home-directory=/home/user --in-memory=true --install=false --launch-browser=true --log=false --token=token in /home/user/.getML/getml-1.5.0-x64-linux...
Launched the getML Engine. The log output will be stored in /home/user/.getML/logs/20240912131357.log.
  Loading pipelines... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


Connected to project 'adventure_works'.

### 1. Loading data

#### 1.1 Download from source

We begin by downloading the data:

In [5]:
conn = getml.database.connect_mysql(
    host="db.relational-data.org",
    dbname="AdventureWorks2014",
    port=3306,
    user="guest",
    password="relational"
)

conn

Connection(dbname='AdventureWorks2014',
           dialect='mysql',
           host='db.relational-data.org',
           port=3306)

In [6]:
def load_if_needed(name):
    """
    Loads the data from the relational learning
    repository, if the data frame has not already
    been loaded.
    """
    if not getml.data.exists(name):
        data_frame = getml.DataFrame.from_db(
            name=name,
            table_name=name,
            conn=conn
        )
        data_frame.save()
    else:
        data_frame = getml.data.load_data_frame(name)
    return data_frame

In [7]:
product = load_if_needed("Product")
sales_order_detail = load_if_needed("SalesOrderDetail")
sales_order_header = load_if_needed("SalesOrderHeader")
sales_order_reason = load_if_needed("SalesOrderHeaderSalesReason")
special_offer = load_if_needed("SpecialOffer")
store = load_if_needed("Store")

In [8]:
product

name      ProductID       MakeFlag   FinishedGoodsFlag   SafetyStockLevel   ...   SellStartDate         SellEndDate  
role   unused_float   unused_float        unused_float       unused_float   ...   unused_string         unused_string
   0              1              0                   0               1000   ...   2008-04-30 00:00:00   NULL         
   1              2              0                   0               1000   ...   2008-04-30 00:00:00   NULL         
   2              3              1                   0                800   ...   2008-04-30 00:00:00   NULL         
   3              4              0                   0                800   ...   2008-04-30 00:00:00   NULL         
   4            316              1                   0                800   ...   2008-04-30 00:00:00   NULL         
                ...            ...                 ...                ...         ...                   ...          
 499            995              1                   1                500   ...   2013-05-30 00:00:00   NULL         
 500            996              1                   1                500   ...   2013-05-30 00:00:00   NULL         
 501            997              1                   1                100   ...   2013-05-30 00:00:00   NULL         
 502            998              1                   1                100   ...   2013-05-30 00:00:00   NULL         
 503            999              1                   1                100   ...   2013-05-30 00:00:00   NULL         

name   DiscontinuedDate   rowguid                            ModifiedDate       
role   unused_string      unused_string                      unused_string      
   0   NULL               694215B7-08F7-4C0D-ACB1-D734BA44...   2014-02-08 09:01:36
   1   NULL               58AE3C20-4F3A-4749-A7D4-D568806C...   2014-02-08 09:01:36
   2   NULL               9C21AED2-5BFA-4F18-BCB8-F11638DC...   2014-02-08 09:01:36
   3   NULL               ECFED6CB-51FF-49B5-B06C-7D8AC834...   2014-02-08 09:01:36
   4   NULL               E73E9750-603B-4131-89F5-3DD15ED5...   2014-02-08 09:01:36
       ...                ...                                ...                
 499   NULL               71AB847F-D091-42D6-B735-7B0C2D82...   2014-02-08 09:01:36
 500   NULL               230C47C5-08B2-4CE3-B706-69C0BDD6...   2014-02-08 09:01:36
 501   NULL               44CE4802-409F-43AB-9B27-CA534218...   2014-02-08 09:01:36
 502   NULL               3DE9A212-1D49-40B6-B10A-F564D981...   2014-02-08 09:01:36
 503   NULL               AE638923-2B67-4679-B90E-ABBAB17D...   2014-02-08 09:01:36


504 rows x 25 columns
memory usage: 0.17 MB
type: getml.DataFrame

In [9]:
sales_order_detail

name   SalesOrderID   SalesOrderDetailID       OrderQty      ProductID   ...   UnitPrice       UnitPriceDiscount
  role   unused_float         unused_float   unused_float   unused_float   ...   unused_string   unused_string    
     0          43659                    1              1            776   ...   2024.9940       0.0000           
     1          43659                    2              3            777   ...   2024.9940       0.0000           
     2          43659                    3              1            778   ...   2024.9940       0.0000           
     3          43659                    4              1            771   ...   2039.9940       0.0000           
     4          43659                    5              1            772   ...   2039.9940       0.0000           
                  ...                  ...            ...            ...         ...             ...              
121312          75122               121313              1            878   ...   21.9800         0.0000           
121313          75122               121314              1            712   ...   8.9900          0.0000           
121314          75123               121315              1            878   ...   21.9800         0.0000           
121315          75123               121316              1            879   ...   159.0000        0.0000           
121316          75123               121317              1            712   ...   8.9900          0.0000           

  name   LineTotal       rowguid                            ModifiedDate       
  role   unused_string   unused_string                      unused_string      
     0   2024.994000     B207C96D-D9E6-402B-8470-2CC176C4...   2011-05-30 22:00:00
     1   6074.982000     7ABB600D-1E77-41BE-9FE5-B9142CFC...   2011-05-30 22:00:00
     2   2024.994000     475CF8C6-49F6-486E-B0AD-AFC6A50C...   2011-05-30 22:00:00
     3   2039.994000     04C4DE91-5815-45D6-8670-F462719F...   2011-05-30 22:00:00
     4   2039.994000     5A74C7D2-E641-438E-A7AC-37BF2328...   2011-05-30 22:00:00
         ...             ...                                ...                
121312   21.980000       8CAD6675-18CC-4F47-8287-97B41A8E...   2014-06-29 22:00:00
121313   8.990000        84F1C363-1C50-4442-BE16-541C59B6...   2014-06-29 22:00:00
121314   21.980000       C18B6476-429F-4BB1-828E-2BE5F82A...   2014-06-29 22:00:00
121315   159.000000      75A89C6A-C60A-47EA-8A52-B52A9C43...   2014-06-29 22:00:00
121316   8.990000        73646D26-0461-450D-8019-2C6C8586...   2014-06-29 22:00:00


121317 rows x 11 columns
memory usage: 21.60 MB
type: getml.DataFrame

In [10]:
sales_order_header

name   SalesOrderID   RevisionNumber         Status   OnlineOrderFlag   ...   Freight         TotalDue        Comment      
 role   unused_float     unused_float   unused_float      unused_float   ...   unused_string   unused_string   unused_string
    0          43659                8              5                 0   ...   616.0984        23153.2339      NULL         
    1          43660                8              5                 0   ...   38.8276         1457.3288       NULL         
    2          43661                8              5                 0   ...   985.5530        36865.8012      NULL         
    3          43662                8              5                 0   ...   867.2389        32474.9324      NULL         
    4          43663                8              5                 0   ...   12.5838         472.3108        NULL         
                 ...              ...            ...               ...         ...             ...             ...          
31460          75119                8              5                 1   ...   1.0570          46.7194         NULL         
31461          75120                8              5                 1   ...   2.1240          93.8808         NULL         
31462          75121                8              5                 1   ...   1.8745          82.8529         NULL         
31463          75122                8              5                 1   ...   0.7743          34.2219         NULL         
31464          75123                8              5                 1   ...   4.7493          209.9169        NULL         

 name   rowguid                            ModifiedDate       
 role   unused_string                      unused_string      
    0   79B65321-39CA-4115-9CBA-8FE0903E...   2011-06-06 22:00:00
    1   738DC42D-D03B-48A1-9822-F95A67EA...   2011-06-06 22:00:00
    2   D91B9131-18A4-4A11-BC3A-90B6F53E...   2011-06-06 22:00:00
    3   4A1ECFC0-CC3A-4740-B028-1C50BB48...   2011-06-06 22:00:00
    4   9B1E7A40-6AE0-4AD3-811C-A6495185...   2011-06-06 22:00:00
        ...                                ...                
31460   9382F1C9-383A-435F-9449-0EECEA21...   2014-07-06 22:00:00
31461   AE6A4FCF-FF73-4CD4-AF2C-5993D00D...   2014-07-06 22:00:00
31462   D7395C0E-00CB-4BFA-A238-0D6A9F49...   2014-07-06 22:00:00
31463   4221035A-4159-492F-AF40-4363A64F...   2014-07-06 22:00:00
31464   D54752FF-2B54-4BE5-95EA-3B72289C...   2014-07-06 22:00:00


31465 rows x 26 columns
memory usage: 12.56 MB
type: getml.DataFrame

In [11]:
sales_order_reason

name   SalesOrderID   SalesReasonID   ModifiedDate       
 role   unused_float    unused_float   unused_string      
    0          43697               5   2011-05-30 22:00:00
    1          43697               9   2011-05-30 22:00:00
    2          43702               5   2011-05-31 22:00:00
    3          43702               9   2011-05-31 22:00:00
    4          43703               5   2011-05-31 22:00:00
                 ...             ...   ...                
27642          75119               1   2014-06-29 22:00:00
27643          75120               1   2014-06-29 22:00:00
27644          75121               1   2014-06-29 22:00:00
27645          75122               1   2014-06-29 22:00:00
27646          75123               1   2014-06-29 22:00:00


27647 rows x 3 columns
memory usage: 1.22 MB
type: getml.DataFrame

In [12]:
special_offer

name   SpecialOfferID         MinQty         MaxQty   Description                        ...   Category     
role     unused_float   unused_float   unused_float   unused_string                      ...   unused_string
   0                1              0           nan    No Discount                        ...   No Discount  
   1                2             11            14    Volume Discount 11 to 14           ...   Reseller     
   2                3             15            24    Volume Discount 15 to 24           ...   Reseller     
   3                4             25            40    Volume Discount 25 to 40           ...   Reseller     
   4                5             41            60    Volume Discount 41 to 60           ...   Reseller     
                  ...            ...            ...   ...                                      ...          
  11               12              0           nan    LL Road Frame Sale                 ...   Reseller     
  12               13              0           nan    Touring-3000 Promotion             ...   Reseller     
  13               14              0           nan    Touring-1000 Promotion             ...   Reseller     
  14               15              0           nan    Half-Price Pedal Sale              ...   Customer     
  15               16              0           nan    Mountain-500 Silver Clearance Sa...   ...   Reseller     

name   StartDate             EndDate               rowguid                            ModifiedDate       
role   unused_string         unused_string         unused_string                      unused_string      
   0   2011-05-01 00:00:00   2014-11-30 00:00:00   0290C4F5-191F-4337-AB6B-0A2DDE03...   2011-03-31 22:00:00
   1   2011-05-31 00:00:00   2014-05-30 00:00:00   D7542EE7-15DB-4541-985C-5CC27AEF...   2011-04-30 22:00:00
   2   2011-05-31 00:00:00   2014-05-30 00:00:00   4BDBCC01-8CF7-40A9-B643-40EC5B71...   2011-04-30 22:00:00
   3   2011-05-31 00:00:00   2014-05-30 00:00:00   504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...   2011-04-30 22:00:00
   4   2011-05-31 00:00:00   2014-05-30 00:00:00   677E1D9D-944F-4E81-90E8-47EB0A82...   2011-04-30 22:00:00
       ...                   ...                   ...                                ...                
  11   2013-05-30 00:00:00   2013-07-14 00:00:00   C0AF1C89-9722-4235-9248-3FBA4D9E...   2013-04-29 22:00:00
  12   2013-05-30 00:00:00   2013-08-29 00:00:00   5061CCE4-E021-45A8-9A75-DFB36CBB...   2013-04-29 22:00:00
  13   2013-05-30 00:00:00   2013-08-29 00:00:00   1AF84A9E-A98C-4BD9-B48F-DC2B8B6B...   2013-04-29 22:00:00
  14   2013-07-14 00:00:00   2013-08-14 00:00:00   03E3594D-6EBB-46A6-B8EE-A9289C0C...   2013-06-13 22:00:00
  15   2014-03-31 00:00:00   2014-05-30 00:00:00   EB7CB484-BCCF-4D2D-BF73-521B2001...   2014-02-28 23:00:00


16 rows x 11 columns
memory usage: 0.00 MB
type: getml.DataFrame

In [13]:
store

name   BusinessEntityID   SalesPersonID   Name                             Demographics                    
role       unused_float    unused_float   unused_string                    unused_string                   
   0                292             279   Next-Door Bike Store             <StoreSurvey xmlns="http://schem...
   1                294             276   Professional Sales and Service   <StoreSurvey xmlns="http://schem...
   2                296             277   Riders Company                   <StoreSurvey xmlns="http://schem...
   3                298             275   The Bike Mechanics               <StoreSurvey xmlns="http://schem...
   4                300             286   Nationwide Supply                <StoreSurvey xmlns="http://schem...
                    ...             ...   ...                              ...                             
 696               1988             282   Retreat Inn                      <StoreSurvey xmlns="http://schem...
 697               1990             281   Technical Parts Manufacturing    <StoreSurvey xmlns="http://schem...
 698               1992             277   Totes & Baskets Company          <StoreSurvey xmlns="http://schem...
 699               1994             277   World of Bikes                   <StoreSurvey xmlns="http://schem...
 700               2051             275   A Bicycle Association            <StoreSurvey xmlns="http://schem...

name   rowguid                            ModifiedDate       
role   unused_string                      unused_string      
   0   A22517E3-848D-4EBE-B9D9-7437F343...   2014-09-12 09:15:07
   1   B50CA50B-C601-4A13-B07E-2C63862D...   2014-09-12 09:15:07
   2   337C3688-1339-4E1A-A08A-B54B2356...   2014-09-12 09:15:07
   3   7894F278-F0C8-4D16-BD75-213FDBF1...   2014-09-12 09:15:07
   4   C3FC9705-A8C4-4F3A-9550-EB2FA4B7...   2014-09-12 09:15:07
       ...                                ...                
 696   EA21EC81-1BFA-4A07-9B4D-73D9852A...   2014-09-12 09:15:07
 697   C8E3C4ED-8F58-4DB2-B600-E0CD11D9...   2014-09-12 09:15:07
 698   CE860B58-643C-4567-BFD8-06E97969...   2014-09-12 09:15:07
 699   0C10F2B6-A13A-440C-9C25-5B28D482...   2014-09-12 09:15:07
 700   82237172-D3FE-4A95-82EF-636F6552...   2014-09-12 09:15:07


701 rows x 6 columns
memory usage: 0.38 MB
type: getml.DataFrame

#### 1.2 Prepare data for getML

getML requires that we define *roles* for each of the columns.

In [14]:
product.set_role("ProductID", getml.data.roles.join_key)
product.set_role(["MakeFlag", "ProductSubcategoryID", "ProductModelID"], getml.data.roles.categorical)
product.set_role(["SafetyStockLevel", "ReorderPoint", "StandardCost", "ListPrice"], getml.data.roles.numerical)

product

name   ProductID   MakeFlag      ProductSubcategoryID   ProductModelID   ...   SellStartDate         SellEndDate  
role    join_key   categorical   categorical            categorical      ...   unused_string         unused_string
   0           1   0             nan                    nan              ...   2008-04-30 00:00:00   NULL         
   1           2   0             nan                    nan              ...   2008-04-30 00:00:00   NULL         
   2           3   1             nan                    nan              ...   2008-04-30 00:00:00   NULL         
   3           4   0             nan                    nan              ...   2008-04-30 00:00:00   NULL         
   4         316   1             nan                    nan              ...   2008-04-30 00:00:00   NULL         
             ...   ...           ...                    ...                    ...                   ...          
 499         995   1             5                      96               ...   2013-05-30 00:00:00   NULL         
 500         996   1             5                      97               ...   2013-05-30 00:00:00   NULL         
 501         997   1             2                      31               ...   2013-05-30 00:00:00   NULL         
 502         998   1             2                      31               ...   2013-05-30 00:00:00   NULL         
 503         999   1             2                      31               ...   2013-05-30 00:00:00   NULL         

name   DiscontinuedDate   rowguid                            ModifiedDate       
role   unused_string      unused_string                      unused_string      
   0   NULL               694215B7-08F7-4C0D-ACB1-D734BA44...   2014-02-08 09:01:36
   1   NULL               58AE3C20-4F3A-4749-A7D4-D568806C...   2014-02-08 09:01:36
   2   NULL               9C21AED2-5BFA-4F18-BCB8-F11638DC...   2014-02-08 09:01:36
   3   NULL               ECFED6CB-51FF-49B5-B06C-7D8AC834...   2014-02-08 09:01:36
   4   NULL               E73E9750-603B-4131-89F5-3DD15ED5...   2014-02-08 09:01:36
       ...                ...                                ...                
 499   NULL               71AB847F-D091-42D6-B735-7B0C2D82...   2014-02-08 09:01:36
 500   NULL               230C47C5-08B2-4CE3-B706-69C0BDD6...   2014-02-08 09:01:36
 501   NULL               44CE4802-409F-43AB-9B27-CA534218...   2014-02-08 09:01:36
 502   NULL               3DE9A212-1D49-40B6-B10A-F564D981...   2014-02-08 09:01:36
 503   NULL               AE638923-2B67-4679-B90E-ABBAB17D...   2014-02-08 09:01:36


504 rows x 25 columns
memory usage: 0.16 MB
type: getml.DataFrame

In [15]:
sales_order_detail.set_role(["SalesOrderID", "SalesOrderDetailID", "ProductID", "SpecialOfferID"], getml.data.roles.join_key)
sales_order_detail.set_role(["OrderQty", "UnitPrice", "UnitPriceDiscount", "LineTotal"], getml.data.roles.numerical)
sales_order_detail.set_role("ModifiedDate", getml.data.roles.time_stamp)

sales_order_detail

name                  ModifiedDate   SalesOrderID   SalesOrderDetailID   ProductID   ...   UnitPrice   UnitPriceDiscount
  role                    time_stamp       join_key             join_key    join_key   ...   numerical           numerical
  unit   time stamp, comparison only                                                   ...                                
     0           2011-05-30 22:00:00          43659                    1         776   ...    2024.994                   0
     1           2011-05-30 22:00:00          43659                    2         777   ...    2024.994                   0
     2           2011-05-30 22:00:00          43659                    3         778   ...    2024.994                   0
     3           2011-05-30 22:00:00          43659                    4         771   ...    2039.994                   0
     4           2011-05-30 22:00:00          43659                    5         772   ...    2039.994                   0
                                 ...            ...                  ...         ...            ...                    ...
121312           2014-06-29 22:00:00          75122               121313         878   ...      21.98                    0
121313           2014-06-29 22:00:00          75122               121314         712   ...       8.99                    0
121314           2014-06-29 22:00:00          75123               121315         878   ...      21.98                    0
121315           2014-06-29 22:00:00          75123               121316         879   ...     159                       0
121316           2014-06-29 22:00:00          75123               121317         712   ...       8.99                    0

  name   LineTotal   CarrierTrackingNumber   rowguid                         
  role   numerical   unused_string           unused_string                   
  unit                                                                       
     0    2024.994   4911-403C-98            B207C96D-D9E6-402B-8470-2CC176C4...
     1    6074.982   4911-403C-98            7ABB600D-1E77-41BE-9FE5-B9142CFC...
     2    2024.994   4911-403C-98            475CF8C6-49F6-486E-B0AD-AFC6A50C...
     3    2039.994   4911-403C-98            04C4DE91-5815-45D6-8670-F462719F...
     4    2039.994   4911-403C-98            5A74C7D2-E641-438E-A7AC-37BF2328...
            ...      ...                     ...                             
121312      21.98    NULL                    8CAD6675-18CC-4F47-8287-97B41A8E...
121313       8.99    NULL                    84F1C363-1C50-4442-BE16-541C59B6...
121314      21.98    NULL                    C18B6476-429F-4BB1-828E-2BE5F82A...
121315     159       NULL                    75A89C6A-C60A-47EA-8A52-B52A9C43...
121316       8.99    NULL                    73646D26-0461-450D-8019-2C6C8586...


121317 rows x 11 columns
memory usage: 14.08 MB
type: getml.DataFrame

In [16]:
sales_order_reason.set_role("SalesOrderID", getml.data.roles.join_key)
sales_order_reason.set_role("SalesReasonID", getml.data.roles.categorical)

sales_order_reason

name   SalesOrderID   SalesReasonID   ModifiedDate       
 role       join_key   categorical     unused_string      
    0          43697   5               2011-05-30 22:00:00
    1          43697   9               2011-05-30 22:00:00
    2          43702   5               2011-05-31 22:00:00
    3          43702   9               2011-05-31 22:00:00
    4          43703   5               2011-05-31 22:00:00
                 ...   ...             ...                
27642          75119   1               2014-06-29 22:00:00
27643          75120   1               2014-06-29 22:00:00
27644          75121   1               2014-06-29 22:00:00
27645          75122   1               2014-06-29 22:00:00
27646          75123   1               2014-06-29 22:00:00


27647 rows x 3 columns
memory usage: 1.00 MB
type: getml.DataFrame

In [17]:
special_offer.set_role(["SpecialOfferID"], getml.data.roles.join_key)
special_offer.set_role(["MinQty", "DiscountPct"], getml.data.roles.numerical)
special_offer.set_role(["Category", "Description", "Type"], getml.data.roles.categorical)
special_offer.set_role(["StartDate", "EndDate"], getml.data.roles.time_stamp)

special_offer

name                     StartDate                       EndDate   SpecialOfferID   Category      ...      MinQty
role                    time_stamp                    time_stamp         join_key   categorical   ...   numerical
unit   time stamp, comparison only   time stamp, comparison only                                  ...            
   0                    2011-05-01                    2014-11-30                1   No Discount   ...           0
   1                    2011-05-31                    2014-05-30                2   Reseller      ...          11
   2                    2011-05-31                    2014-05-30                3   Reseller      ...          15
   3                    2011-05-31                    2014-05-30                4   Reseller      ...          25
   4                    2011-05-31                    2014-05-30                5   Reseller      ...          41
                               ...                           ...              ...   ...                       ...
  11                    2013-05-30                    2013-07-14               12   Reseller      ...           0
  12                    2013-05-30                    2013-08-29               13   Reseller      ...           0
  13                    2013-05-30                    2013-08-29               14   Reseller      ...           0
  14                    2013-07-14                    2013-08-14               15   Customer      ...           0
  15                    2014-03-31                    2014-05-30               16   Reseller      ...           0

name   DiscountPct         MaxQty   rowguid                            ModifiedDate       
role     numerical   unused_float   unused_string                      unused_string      
unit                                                                                      
   0          0              nan    0290C4F5-191F-4337-AB6B-0A2DDE03...   2011-03-31 22:00:00
   1          0.02            14    D7542EE7-15DB-4541-985C-5CC27AEF...   2011-04-30 22:00:00
   2          0.05            24    4BDBCC01-8CF7-40A9-B643-40EC5B71...   2011-04-30 22:00:00
   3          0.1             40    504B5E85-8F3F-4EBC-9E1D-C1BC5DEA...   2011-04-30 22:00:00
   4          0.15            60    677E1D9D-944F-4E81-90E8-47EB0A82...   2011-04-30 22:00:00
             ...              ...   ...                                ...                
  11          0.35           nan    C0AF1C89-9722-4235-9248-3FBA4D9E...   2013-04-29 22:00:00
  12          0.15           nan    5061CCE4-E021-45A8-9A75-DFB36CBB...   2013-04-29 22:00:00
  13          0.2            nan    1AF84A9E-A98C-4BD9-B48F-DC2B8B6B...   2013-04-29 22:00:00
  14          0.5            nan    03E3594D-6EBB-46A6-B8EE-A9289C0C...   2013-06-13 22:00:00
  15          0.4            nan    EB7CB484-BCCF-4D2D-BF73-521B2001...   2014-02-28 23:00:00


16 rows x 11 columns
memory usage: 0.00 MB
type: getml.DataFrame

In [18]:
store.set_role(["SalesPersonID"], getml.data.roles.join_key)

store["test"] = store["ModifiedDate"].update(getml.data.random() > 0.5, "NULL")

store.set_role(["SalesPersonID"], getml.data.roles.join_key)
store.set_role(["test"], getml.data.roles.time_stamp)

store

name                          test   SalesPersonID   BusinessEntityID   Name                          
role                    time_stamp        join_key       unused_float   unused_string                 
unit   time stamp, comparison only                                                                    
   0           2014-09-12 09:15:07             279                292   Next-Door Bike Store          
   1                          NULL             276                294   Professional Sales and Service
   2           2014-09-12 09:15:07             277                296   Riders Company                
   3                          NULL             275                298   The Bike Mechanics            
   4                          NULL             286                300   Nationwide Supply             
                               ...             ...                ...   ...                           
 696                          NULL             282               1988   Retreat Inn                   
 697           2014-09-12 09:15:07             281               1990   Technical Parts Manufacturing 
 698                          NULL             277               1992   Totes & Baskets Company       
 699                          NULL             277               1994   World of Bikes                
 700           2014-09-12 09:15:07             275               2051   A Bicycle Association         

name   Demographics                       rowguid                            ModifiedDate       
role   unused_string                      unused_string                      unused_string      
unit                                                                                            
   0   <StoreSurvey xmlns="http://schem...   A22517E3-848D-4EBE-B9D9-7437F343...   2014-09-12 09:15:07
   1   <StoreSurvey xmlns="http://schem...   B50CA50B-C601-4A13-B07E-2C63862D...   2014-09-12 09:15:07
   2   <StoreSurvey xmlns="http://schem...   337C3688-1339-4E1A-A08A-B54B2356...   2014-09-12 09:15:07
   3   <StoreSurvey xmlns="http://schem...   7894F278-F0C8-4D16-BD75-213FDBF1...   2014-09-12 09:15:07
   4   <StoreSurvey xmlns="http://schem...   C3FC9705-A8C4-4F3A-9550-EB2FA4B7...   2014-09-12 09:15:07
       ...                                ...                                ...                
 696   <StoreSurvey xmlns="http://schem...   EA21EC81-1BFA-4A07-9B4D-73D9852A...   2014-09-12 09:15:07
 697   <StoreSurvey xmlns="http://schem...   C8E3C4ED-8F58-4DB2-B600-E0CD11D9...   2014-09-12 09:15:07
 698   <StoreSurvey xmlns="http://schem...   CE860B58-643C-4567-BFD8-06E97969...   2014-09-12 09:15:07
 699   <StoreSurvey xmlns="http://schem...   0C10F2B6-A13A-440C-9C25-5B28D482...   2014-09-12 09:15:07
 700   <StoreSurvey xmlns="http://schem...   82237172-D3FE-4A95-82EF-636F6552...   2014-09-12 09:15:07


701 rows x 7 columns
memory usage: 0.38 MB
type: getml.DataFrame

In [19]:
sales_order_header["SalesPersonIDCat"] = sales_order_header["SalesPersonID"]
sales_order_header["TerritoryIDCat"] = sales_order_header["TerritoryID"]

sales_order_header.set_role(["CustomerID", "SalesOrderID", "SalesPersonID", "TerritoryID"], getml.data.roles.join_key)
sales_order_header.set_role(
    ["RevisionNumber", "OnlineOrderFlag", "SalesPersonIDCat", "TerritoryIDCat", "ShipMethodID"], 
    getml.data.roles.categorical)
sales_order_header.set_role(["SubTotal", "TaxAmt", "Freight", "TotalDue"], getml.data.roles.numerical)
sales_order_header.set_role(["OrderDate", "DueDate", "ShipDate", "ModifiedDate"], getml.data.roles.time_stamp)

sales_order_header

name                     OrderDate                       DueDate                      ShipDate                  ModifiedDate
 role                    time_stamp                    time_stamp                    time_stamp                    time_stamp
 unit   time stamp, comparison only   time stamp, comparison only   time stamp, comparison only   time stamp, comparison only
    0           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    1           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    2           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    3           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    4           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
                                ...                           ...                           ...                           ...
31460           2014-06-29 22:00:00                    2014-07-12                    2014-07-07           2014-07-06 22:00:00
31461           2014-06-29 22:00:00                    2014-07-12                    2014-07-07           2014-07-06 22:00:00
31462           2014-06-29 22:00:00                    2014-07-12                    2014-07-07           2014-07-06 22:00:00
31463           2014-06-29 22:00:00                    2014-07-12                    2014-07-07           2014-07-06 22:00:00
31464           2014-06-29 22:00:00                    2014-07-12                    2014-07-07           2014-07-06 22:00:00

 name   ...   PurchaseOrderNumber   AccountNumber    CreditCardApprovalCode   Comment         rowguid                         
 role   ...   unused_string         unused_string    unused_string            unused_string   unused_string                   
 unit   ...                                                                                                                   
    0   ...   PO522145787           10-4020-000676   105041Vi84182            NULL         
    1   ...   PO18850127500         10-4020-000117   115213Vi29411            NULL         
    2   ...   PO18473189620         10-4020-000442   85274Vi6854              NULL         
    3   ...   PO18444174044         10-4020-000227   125295Vi53935            NULL         
    4   ...   PO18009186470         10-4020-000510   45303Vi22691             NULL         
              ...                   ...              ...                      ...             ...                             
31460   ...   NULL                  10-4030-011981   429826Vi35166            NULL         
31461   ...   NULL                  10-4030-018749   929849Vi46003            NULL         
31462   ...   NULL                  10-4030-015251   529864Vi73738            NULL         
31463   ...   NULL                  10-4030-015868   330022Vi97312            NULL         
31464   ...   NULL                  10-4030-018759   230370Vi51970            NULL         

    0   79B65321-39CA-4115-9CBA-8FE0903E...
    1   738DC42D-D03B-48A1-9822-F95A67EA...
    2   D91B9131-18A4-4A11-BC3A-90B6F53E...
    3   4A1ECFC0-CC3A-4740-B028-1C50BB48...
    4   9B1E7A40-6AE0-4AD3-811C-A6495185...
31460   9382F1C9-383A-435F-9449-0EECEA21...
31461   AE6A4FCF-FF73-4CD4-AF2C-5993D00D...
31462   D7395C0E-00CB-4BFA-A238-0D6A9F49...
31463   4221035A-4159-492F-AF40-4363A64F...
31464   D54752FF-2B54-4BE5-95EA-3B72289C...


31465 rows x 28 columns
memory usage: 8.34 MB
type: getml.DataFrame

We must also define customer churn. In this case, we define customer churn as a customer not making another purchase within 180 days of his or her last purchase.

Thus, the churn variable is defined as follows:

- 0, if another purchase by the same customer has been made within 180 days after `OrderDate`
- 1, if no purchase by the same customer has been made within 180 days after `OrderDate`
- NULL, if `max(OrderDate) - OrderDate <= 180 days`

NULL targets can not be used in our analysis.

In [20]:
sales_order_header_pd = sales_order_header[["OrderDate", "CustomerID", "SalesOrderID"]].to_pandas()

repeat_purchases = sales_order_header_pd.merge(
    sales_order_header_pd[["OrderDate", "CustomerID"]],
    on="CustomerID",
    how="left",
)

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] > repeat_purchases["OrderDate_x"]
]

repeat_purchases = repeat_purchases[
    repeat_purchases["OrderDate_y"] - repeat_purchases["OrderDate_x"] > pd.Timedelta('180 days')
]

repeat_purchases.groupby("SalesOrderID", as_index=False).aggregate({"CustomerID": "max"})

repeat_purchase_ids = {sid: True for sid in repeat_purchases["SalesOrderID"]}

cut_off_date = max(sales_order_header_pd["OrderDate"]) - pd.Timedelta('180 days')

churn = np.asarray([
    np.nan if order_date >= cut_off_date else 0 if order_id in repeat_purchase_ids else 1 
    for (order_date, order_id) in zip(sales_order_header_pd["OrderDate"], sales_order_header_pd["SalesOrderID"])
])

sales_order_header["churn"] = churn

sales_order_header = sales_order_header[~sales_order_header.churn.is_nan()].to_df("SalesOrderHeaderRefined")

sales_order_header.set_role("churn", getml.data.roles.target)

sales_order_header

name                     OrderDate                       DueDate                      ShipDate                  ModifiedDate
 role                    time_stamp                    time_stamp                    time_stamp                    time_stamp
 unit   time stamp, comparison only   time stamp, comparison only   time stamp, comparison only   time stamp, comparison only
    0           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    1           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    2           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    3           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
    4           2011-05-30 22:00:00                    2011-06-12                    2011-06-07           2011-06-06 22:00:00
                                ...                           ...                           ...                           ...
19699           2013-12-30 23:00:00                    2014-01-12                    2014-01-07           2014-01-06 23:00:00
19700           2013-12-30 23:00:00                    2014-01-12                    2014-01-07           2014-01-06 23:00:00
19701           2013-12-30 23:00:00                    2014-01-12                    2014-01-07           2014-01-06 23:00:00
19702           2013-12-30 23:00:00                    2014-01-12                    2014-01-07           2014-01-06 23:00:00
19703           2013-12-30 23:00:00                    2014-01-12                    2014-01-07           2014-01-06 23:00:00

 name   ...   PurchaseOrderNumber   AccountNumber    CreditCardApprovalCode   Comment         rowguid                         
 role   ...   unused_string         unused_string    unused_string            unused_string   unused_string                   
 unit   ...                                                                                                                   
    0   ...   PO522145787           10-4020-000676   105041Vi84182            NULL         
    1   ...   PO18850127500         10-4020-000117   115213Vi29411            NULL         
    2   ...   PO18473189620         10-4020-000442   85274Vi6854              NULL         
    3   ...   PO18444174044         10-4020-000227   125295Vi53935            NULL         
    4   ...   PO18009186470         10-4020-000510   45303Vi22691             NULL         
              ...                   ...              ...                      ...             ...                             
19699   ...   NULL                  10-4030-020826   1142084Vi17039           NULL         
19700   ...   NULL                  10-4030-024114   NULL                     NULL         
19701   ...   NULL                  10-4030-011417   242387Vi34223            NULL         
19702   ...   NULL                  10-4030-018125   1242859Vi61993           NULL         
19703   ...   NULL                  10-4030-015692   242864Vi83167            NULL         

    0   79B65321-39CA-4115-9CBA-8FE0903E...
    1   738DC42D-D03B-48A1-9822-F95A67EA...
    2   D91B9131-18A4-4A11-BC3A-90B6F53E...
    3   4A1ECFC0-CC3A-4740-B028-1C50BB48...
    4   9B1E7A40-6AE0-4AD3-811C-A6495185...
19699   41278FBB-3DD8-488B-AEA5-8BF4A6F1...
19700   749532D4-BFF7-4FEC-9F77-396C4A96...
19701   F92FA2A3-73E1-4DD0-987C-99D1C87E...
19702   3A3758BF-CDFA-4740-9104-87BD5A08...
19703   3151D31F-4020-41B2-87F3-B6732869...


19704 rows x 29 columns
memory usage: 5.39 MB
type: getml.DataFrame

In [21]:
split = getml.data.split.random(train=0.8, test=0.2)
split

0   train
 1   train
 2   train
 3   test 
 4   train
     ...  


infinite number of  rows
type: StringColumnView

In [22]:
container = getml.data.Container(population=sales_order_header, split=split)

container.add(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
)

container

population
    subset   name                       rows   type
0   test     SalesOrderHeaderRefined    3879   View
1   train    SalesOrderHeaderRefined   15825   View

peripheral
    alias                name                            rows   type     
0   product              Product                          504   DataFrame
1   sales_order_detail   SalesOrderDetail              121317   DataFrame
2   sales_order_header   SalesOrderHeaderRefined        19704   DataFrame
3   sales_order_reason   SalesOrderHeaderSalesReason    27647   DataFrame
4   special_offer        SpecialOffer                      16   DataFrame
5   store                Store                            701   DataFrame

### 2. Predictive modeling

We loaded the data and defined the roles and units. Next, we create a getML pipeline for relational learning.

#### 2.1 Define relational model

In [23]:
dm = getml.data.DataModel(sales_order_header.to_placeholder("population"))

dm.add(getml.data.to_placeholder(
    product=product,
    sales_order_detail=sales_order_detail, 
    sales_order_header=sales_order_header, 
    sales_order_reason=sales_order_reason,
    special_offer=special_offer,
    store=store,
))

dm.population.join(
    dm.sales_order_header,
    on="CustomerID",
    time_stamps="OrderDate",
    lagged_targets=True,
    horizon=getml.data.time.days(1),
)

dm.population.join(
    dm.sales_order_detail,
    on="SalesOrderID",
)

dm.population.join(
    dm.sales_order_reason,
    on="SalesOrderID", 
)

dm.population.join(
    dm.store,
    on="SalesPersonID", 
)

dm.sales_order_detail.join(
    dm.product,
    on="ProductID",
    relationship=getml.data.relationship.many_to_one,
)

dm.sales_order_detail.join(
    dm.special_offer,
    on="SpecialOfferID",
    relationship=getml.data.relationship.many_to_one,
)

dm

population:
  columns:
  - RevisionNumber: categorical
  - OnlineOrderFlag: categorical
  - SalesPersonIDCat: categorical
  - TerritoryIDCat: categorical
  - ShipMethodID: categorical
  - ...

  joins:
  - right: 'sales_order_header'
    on: (population.CustomerID, sales_order_header.CustomerID)
    time_stamps: (population.OrderDate, sales_order_header.OrderDate)
    relationship: 'many-to-many'
    horizon: 86400.0
    lagged_targets: True
  - right: 'sales_order_detail'
    on: (population.SalesOrderID, sales_order_detail.SalesOrderID)
    relationship: 'many-to-many'
    lagged_targets: False
  - right: 'sales_order_reason'
    on: (population.SalesOrderID, sales_order_reason.SalesOrderID)
    relationship: 'many-to-many'
    lagged_targets: False
  - right: 'store'
    on: (population.SalesPersonID, store.SalesPersonID)
    relationship: 'many-to-many'
    lagged_targets: False

sales_order_header:
  columns:
  - RevisionNumber: categorical
  - OnlineOrderFlag: categorical
  - SalesPersonIDCat: categorical
  - TerritoryIDCat: categorical
  - ShipMethodID: categorical
  - ...

sales_order_detail:
  columns:
  - SalesOrderID: join_key
  - SalesOrderDetailID: join_key
  - ProductID: join_key
  - SpecialOfferID: join_key
  - OrderQty: numerical
  - ...

  joins:
  - right: 'product'
    on: (sales_order_detail.ProductID, product.ProductID)
    relationship: 'many-to-one'
    lagged_targets: False
  - right: 'special_offer'
    on: (sales_order_detail.SpecialOfferID, special_offer.SpecialOfferID)
    relationship: 'many-to-one'
    lagged_targets: False

product:
  columns:
  - MakeFlag: categorical
  - ProductSubcategoryID: categorical
  - ProductModelID: categorical
  - ProductID: join_key
  - SafetyStockLevel: numerical
  - ...

special_offer:
  columns:
  - Category: categorical
  - Description: categorical
  - Type: categorical
  - SpecialOfferID: join_key
  - MinQty: numerical
  - ...

sales_order_reason:
  columns:
  - SalesReasonID: categorical
  - SalesOrderID: join_key
  - ModifiedDate: unused_string

store:
  columns:
  - SalesPersonID: join_key
  - test: time_stamp
  - BusinessEntityID: unused_float
  - Name: unused_string
  - Demographics: unused_string
  - ...

#### 2.2 getML pipeline

<!-- #### 2.1.1  -->
__Set-up the feature learner & predictor__

In [24]:
seasonal = getml.preprocessors.Seasonal()

mapping = getml.preprocessors.Mapping()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
    num_features=400,
)

relboost = getml.feature_learning.Relboost(
    loss_function=getml.feature_learning.loss_functions.CrossEntropyLoss,
    num_threads=1,    
)

predictor = getml.predictors.XGBoostClassifier(n_jobs=1)

__Build the pipeline__

In [25]:
pipe1 = getml.Pipeline(
    tags=['fast_prop'],
    data_model=dm,
    preprocessors=[mapping],
    feature_learners=[fast_prop],
    predictors=[predictor],
    include_categorical=True,
)

pipe1

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop'])

In [26]:
pipe2 = getml.Pipeline(
    tags=['relboost'],
    data_model=dm,
    preprocessors=[seasonal, mapping],
    feature_learners=[relboost],
    predictors=[predictor],
    include_categorical=True,
)

pipe2

Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost'])

#### 2.3 Model training

In [27]:
pipe1.check(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.

type   label                    message                         
0   INFO   FOREIGN KEYS NOT FOUND   When joining POPULATION__STAGING...
1   INFO   FOREIGN KEYS NOT FOUND   When joining POPULATION__STAGING...

In [28]:
pipe1.fit(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Trying 710 features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:02
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:01
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:15


Trained pipeline.

Time taken: 0:00:18.910607.



Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Mapping'],
         share_selected_features=0.5,
         tags=['fast_prop', 'container-OeYnEZ'])

In [29]:
pipe2.check(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.

type   label                    message                         
0   INFO   FOREIGN KEYS NOT FOUND   When joining POPULATION__STAGING...
1   INFO   FOREIGN KEYS NOT FOUND   When joining POPULATION__STAGING...

In [30]:
pipe2.fit(container.train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 2 issues labeled INFO and 0 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Training features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:37
  Relboost: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:05
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:03


Trained pipeline.

Time taken: 0:00:47.069322.



Pipeline(data_model='population',
         feature_learners=['Relboost'],
         feature_selectors=[],
         include_categorical=True,
         loss_function='CrossEntropyLoss',
         peripheral=['product', 'sales_order_detail', 'sales_order_header',
                     'sales_order_reason', 'special_offer', 'store'],
         predictors=['XGBoostClassifier'],
         preprocessors=['Seasonal', 'Mapping'],
         share_selected_features=0.5,
         tags=['relboost', 'container-OeYnEZ'])

#### 2.4 Model evaluation

In [31]:
fastprop_score = pipe1.score(container.test)
fastprop_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  FastProp: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


date time             set used   target   accuracy       auc   cross entropy
0   2024-09-12 13:01:58   train      churn      0.9187    0.975           0.2106
1   2024-09-12 13:02:48   test       churn      0.9142    0.9723          0.2199

In [32]:
relboost_score = pipe2.score(container.test)
relboost_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Relboost: Building features... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:01


date time             set used   target   accuracy       auc   cross entropy
0   2024-09-12 13:02:47   train      churn      0.9323    0.983           0.169 
1   2024-09-12 13:02:51   test       churn      0.9286    0.9788          0.1888

#### 2.5 featuretools

In [33]:
population_train_pd = container.train.population.drop(container.train.population.roles.unused).to_pandas()
population_test_pd = container.test.population.drop(container.train.population.roles.unused).to_pandas()

In [34]:
product_pd = product.drop(product.roles.unused).to_pandas()
sales_order_detail_pd = sales_order_detail.drop(sales_order_detail.roles.unused).to_pandas()
sales_order_header_pd = sales_order_header.drop(sales_order_header.roles.unused).to_pandas()
sales_order_reason_pd = sales_order_reason.drop(sales_order_reason.roles.unused).to_pandas()
special_offer_pd = special_offer.drop(special_offer.roles.unused).to_pandas()
store_pd = store.drop(store.roles.unused).to_pandas()

featuretools does not support many-to-one joins. Therefore, we must manually merge `sales_order_detail_pd`, `product_pd` and `special_offer_pd`.

In [35]:
sales_order_detail_pd = sales_order_detail_pd.merge(
    product_pd,
    on="ProductID",
    how="left",
)

sales_order_detail_pd = sales_order_detail_pd.merge(
    special_offer_pd,
    on="SpecialOfferID",
    how="left",
)

del sales_order_detail_pd["SalesOrderDetailID"]
del sales_order_detail_pd["ProductID"]
del sales_order_detail_pd["SpecialOfferID"]

sales_order_detail_pd

SalesOrderID  OrderQty  UnitPrice  UnitPriceDiscount  LineTotal  \
0             43659       1.0   2024.994                0.0   2024.994   
1             43659       3.0   2024.994                0.0   6074.982   
2             43659       1.0   2024.994                0.0   2024.994   
3             43659       1.0   2039.994                0.0   2039.994   
4             43659       1.0   2039.994                0.0   2039.994   
...             ...       ...        ...                ...        ...   
121312        75122       1.0     21.980                0.0     21.980   
121313        75122       1.0      8.990                0.0      8.990   
121314        75123       1.0     21.980                0.0     21.980   
121315        75123       1.0    159.000                0.0    159.000   
121316        75123       1.0      8.990                0.0      8.990   

              ModifiedDate MakeFlag ProductSubcategoryID ProductModelID  \
0      2011-05-30 22:00:00        1                    1             19   
1      2011-05-30 22:00:00        1                    1             19   
2      2011-05-30 22:00:00        1                    1             19   
3      2011-05-30 22:00:00        1                    1             19   
4      2011-05-30 22:00:00        1                    1             19   
...                    ...      ...                  ...            ...   
121312 2014-06-29 22:00:00        0                   30            121   
121313 2014-06-29 22:00:00        0                   19              2   
121314 2014-06-29 22:00:00        0                   30            121   
121315 2014-06-29 22:00:00        0                   27            122   
121316 2014-06-29 22:00:00        0                   19              2   

        SafetyStockLevel  ReorderPoint  StandardCost  ListPrice     Category  \
0                  100.0          75.0     1898.0944    3374.99  No Discount   
1                  100.0          75.0     1898.0944    3374.99  No Discount   
2                  100.0          75.0     1898.0944    3374.99  No Discount   
3                  100.0          75.0     1912.1544    3399.99  No Discount   
4                  100.0          75.0     1912.1544    3399.99  No Discount   
...                  ...           ...           ...        ...          ...   
121312               4.0           3.0        8.2205      21.98  No Discount   
121313               4.0           3.0        6.9223       8.99  No Discount   
121314               4.0           3.0        8.2205      21.98  No Discount   
121315               4.0           3.0       59.4660     159.00  No Discount   
121316               4.0           3.0        6.9223       8.99  No Discount   

        Description         Type  MinQty  DiscountPct  StartDate    EndDate  
0       No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
1       No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
2       No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
3       No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
4       No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
...             ...          ...     ...          ...        ...        ...  
121312  No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
121313  No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
121314  No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
121315  No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  
121316  No Discount  No Discount     0.0          0.0 2011-05-01 2014-11-30  

[121317 rows x 20 columns]

In [36]:
def prepare_sales_order_header(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["CustomerID", "OrderDate", "SalesOrderID"]],
        on="CustomerID"
    )

    peripheral_new = peripheral_new[
        peripheral_new["OrderDate_x"] < peripheral_new["OrderDate_y"]
    ]
    
    del peripheral_new["SalesOrderID_x"]
    del peripheral_new["OrderDate_y"]
    del peripheral_new["CustomerID"]
    del peripheral_new["SalesPersonIDCat"]
    del peripheral_new["TerritoryIDCat"]
    
    return peripheral_new.rename(columns={"OrderDate_x": "OrderDate", "SalesOrderID_y": "SalesOrderID"})

In [37]:
def prepare_store(peripheral_pd, train_or_test):
    """
    Helper function that imitates the behavior of 
    the data model defined above.
    """
    peripheral_new = peripheral_pd.merge(
        train_or_test[["SalesPersonID", "SalesOrderID"]],
        on="SalesPersonID"
    )

    return peripheral_new

In [38]:
store_train_pd = prepare_store(store_pd, population_train_pd)
store_test_pd = prepare_store(store_pd, population_test_pd)
store_train_pd

SalesPersonID                test SalesOrderID
0                279 2014-09-12 09:15:07        43659
1                279 2014-09-12 09:15:07        43660
2                279 2014-09-12 09:15:07        43681
3                279 2014-09-12 09:15:07        43685
4                279 2014-09-12 09:15:07        43695
...              ...                 ...          ...
142427           275 2014-09-12 09:15:07        63252
142428           275 2014-09-12 09:15:07        63258
142429           275 2014-09-12 09:15:07        63262
142430           275 2014-09-12 09:15:07        63280
142431           275 2014-09-12 09:15:07        63285

[142432 rows x 3 columns]

In [39]:
sales_order_header_train_pd = prepare_sales_order_header(sales_order_header_pd, population_train_pd)
sales_order_header_test_pd = prepare_sales_order_header(sales_order_header_pd, population_test_pd)
sales_order_header_train_pd

RevisionNumber OnlineOrderFlag ShipMethodID SalesPersonID TerritoryID  \
1                  8               0            5           279           5   
2                  8               0            5           279           5   
3                  8               0            5           279           5   
4                  8               0            5           279           5   
5                  8               0            5           279           5   
...              ...             ...          ...           ...         ...   
38640              8               1            1          None           6   
38914              8               1            1          None           6   
39079              8               1            1          None           6   
39116              8               1            1          None           6   
39201              8               1            1          None           6   

         SubTotal     TaxAmt   Freight    TotalDue  churn           OrderDate  \
1      20565.6206  1971.5149  616.0984  23153.2339    0.0 2011-05-30 22:00:00   
2      20565.6206  1971.5149  616.0984  23153.2339    0.0 2011-05-30 22:00:00   
3      20565.6206  1971.5149  616.0984  23153.2339    0.0 2011-05-30 22:00:00   
4      20565.6206  1971.5149  616.0984  23153.2339    0.0 2011-05-30 22:00:00   
5      20565.6206  1971.5149  616.0984  23153.2339    0.0 2011-05-30 22:00:00   
...           ...        ...       ...         ...    ...                 ...   
38640     21.4900     1.7192    0.5373     23.7465    0.0 2013-12-21 23:00:00   
38914     49.4800     3.9584    1.2370     54.6754    1.0 2013-12-24 23:00:00   
39079     34.9900     2.7992    0.8748     38.6640    1.0 2013-12-26 23:00:00   
39116    108.4600     8.6768    2.7115    119.8483    1.0 2013-12-26 23:00:00   
39201     53.9900     4.3192    1.3498     59.6590    0.0 2013-12-27 23:00:00   

         DueDate   ShipDate        ModifiedDate SalesOrderID  
1     2011-06-12 2011-06-07 2011-06-06 22:00:00        44305  
2     2011-06-12 2011-06-07 2011-06-06 22:00:00        45061  
3     2011-06-12 2011-06-07 2011-06-06 22:00:00        45779  
4     2011-06-12 2011-06-07 2011-06-06 22:00:00        46604  
5     2011-06-12 2011-06-07 2011-06-06 22:00:00        47693  
...          ...        ...                 ...          ...  
38640 2014-01-03 2013-12-29 2013-12-28 23:00:00        62961  
38914 2014-01-06 2014-01-01 2013-12-31 23:00:00        62875  
39079 2014-01-08 2014-01-03 2014-01-02 23:00:00        63307  
39116 2014-01-08 2014-01-03 2014-01-02 23:00:00        63307  
39201 2014-01-09 2014-01-04 2014-01-03 23:00:00        63003  

[12271 rows x 15 columns]

In [40]:
del population_train_pd["CustomerID"]
del population_train_pd["SalesPersonIDCat"]

del population_test_pd["CustomerID"]
del population_test_pd["SalesPersonIDCat"]

featuretools's dataframes are similar for getML's container class.
featuretools uses woodwork for typing information, which will be guessed automatically, but can also be provided manually.

In [41]:
def add_index(df):
    df.insert(0, "index", range(len(df)))

population_pd_logical_types = {
    'RevisionNumber': ww.logical_types.Categorical,
    'OnlineOrderFlag': ww.logical_types.Categorical,
    'ShipMethodID': ww.logical_types.Integer,
    'TerritoryIDCat': ww.logical_types.Integer,
    'SalesOrderID': ww.logical_types.Integer,
    'SalesPersonID': ww.logical_types.IntegerNullable,
    'TerritoryID': ww.logical_types.IntegerNullable,
    'SubTotal': ww.logical_types.Double,
    'TaxAmt': ww.logical_types.Double,
    'Freight': ww.logical_types.Double,
    'TotalDue': ww.logical_types.Double,
    'churn': ww.logical_types.Categorical,
    'OrderDate': ww.logical_types.Datetime,
    'DueDate': ww.logical_types.Datetime,
    'ShipDate': ww.logical_types.Datetime,
    'ModifiedDate': ww.logical_types.Datetime
}
population_train_pd.ww.init(logical_types=population_pd_logical_types, index="SalesOrderID", name="population")
population_test_pd.ww.init(logical_types=population_pd_logical_types, index="SalesOrderID", name="population")

sales_order_detail_pd_logical_types = {
    'index': ww.logical_types.Integer,
    'SalesOrderID': ww.logical_types.Integer,
    'OrderQty': ww.logical_types.Integer,
    'UnitPrice': ww.logical_types.Double,
    'UnitPriceDiscount': ww.logical_types.Double,
    'LineTotal': ww.logical_types.Double,
    'ModifiedDate': ww.logical_types.Datetime,
    # merged with product_pd
    'MakeFlag': ww.logical_types.Categorical,
    'ProductSubcategoryID': ww.logical_types.Categorical,
    'ProductModelID': ww.logical_types.Integer,
    'ReorderPoint': ww.logical_types.Integer,
    'StandardCost': ww.logical_types.Double,
    'ListPrice': ww.logical_types.Double,
    # merged with special_offer_pd
    'Category': ww.logical_types.Categorical,
    'Description': ww.logical_types.Categorical,
    'Type': ww.logical_types.Categorical,
    'MinQty': ww.logical_types.Integer,
    'DiscountPct': ww.logical_types.Double,
    'StartDate': ww.logical_types.Datetime,
    'EndDate': ww.logical_types.Datetime
}
add_index(sales_order_detail_pd)
sales_order_detail_pd.ww.init(logical_types=sales_order_detail_pd_logical_types, index="index", name="sales_order_detail")

sales_order_header_pd_logical_types = {
    'index': ww.logical_types.Integer,
    'RevisionNumber': ww.logical_types.Categorical,
    'OnlineOrderFlag': ww.logical_types.Categorical,
    'ShipMethodID': ww.logical_types.Integer,
    'SalesOrderID': ww.logical_types.Integer,
    'SalesPersonID': ww.logical_types.IntegerNullable,
    'TerritoryID': ww.logical_types.IntegerNullable,
    'SubTotal': ww.logical_types.Double,
    'TaxAmt': ww.logical_types.Double,
    'Freight': ww.logical_types.Double,
    'TotalDue': ww.logical_types.Double,
    'churn': ww.logical_types.Categorical,
    'OrderDate': ww.logical_types.Datetime,
    'DueDate': ww.logical_types.Datetime,
    'ShipDate': ww.logical_types.Datetime,
    'ModifiedDate': ww.logical_types.Datetime
}
add_index(sales_order_header_train_pd)
sales_order_header_train_pd.ww.init(logical_types=sales_order_header_pd_logical_types, index="index", name="sales_order_header")
add_index(sales_order_header_test_pd)
sales_order_header_test_pd.ww.init(logical_types=sales_order_header_pd_logical_types, index="index", name="sales_order_header")

sales_order_reason_pd_logical_types = {
    'index': ww.logical_types.Integer,
    'SalesReasonID': ww.logical_types.Categorical,
    'SalesOrderID': ww.logical_types.Integer
}
add_index(sales_order_reason_pd)
sales_order_reason_pd.ww.init(logical_types=sales_order_reason_pd_logical_types, index="index", name="sales_order_reason")

store_pd_logical_types = {
    'index': ww.logical_types.Integer,
    'SalesPersonID': ww.logical_types.IntegerNullable,
    'test': ww.logical_types.Datetime,
    'SalesOrderID': ww.logical_types.Integer
}
add_index(store_train_pd)
store_train_pd.ww.init(logical_types=store_pd_logical_types, index="index", name="store")
add_index(store_test_pd)
store_test_pd.ww.init(logical_types=store_pd_logical_types, index="index", name="store")

In [42]:
dataframes_train = {
    "population" : (population_train_pd, ),
    "sales_order_header": (sales_order_header_train_pd, ),
    "sales_order_detail": (sales_order_detail_pd, ),
    "sales_order_reason": (sales_order_reason_pd, ),
    "store": (store_train_pd, ),
}

In [43]:
dataframes_test = {
    "population" : (population_test_pd, ),
    "sales_order_header": (sales_order_header_test_pd, ),
    "sales_order_detail": (sales_order_detail_pd, ),
    "sales_order_reason": (sales_order_reason_pd, ),
    "store": (store_test_pd, ),
}

featuretools's relationships are similar for getML's data model.

In [44]:
relationships = [
    ("population", "SalesOrderID", "sales_order_header", "SalesOrderID"),
    ("population", "SalesOrderID", "sales_order_detail", "SalesOrderID"),
    ("population", "SalesOrderID", "sales_order_reason", "SalesOrderID"),
    ("population", "SalesOrderID", "store", "SalesOrderID"),
]

In [45]:
featuretools_train_pd = featuretools.dfs(
    dataframes=dataframes_train,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [46]:
featuretools_test_pd = featuretools.dfs(
    dataframes=dataframes_test,
    relationships=relationships,
    target_dataframe_name="population")[0]

In [47]:
featuretools_train_pd

RevisionNumber OnlineOrderFlag  TerritoryIDCat  ShipMethodID  \
SalesOrderID                                                                
43659                     8               0               5             5   
43660                     8               0               5             5   
43661                     8               0               6             5   
43663                     8               0               4             5   
43664                     8               0               1             5   
...                     ...             ...             ...           ...   
63358                     8               1               7             1   
63359                     8               1              10             1   
63360                     8               1               7             1   
63361                     8               1               8             1   
63362                     8               1              10             1   

              SalesPersonID  TerritoryID    SubTotal     TaxAmt   Freight  \
SalesOrderID                                                                
43659                   279            5  20565.6206  1971.5149  616.0984   
43660                   279            5   1294.2529   124.2483   38.8276   
43661                   282            6  32726.4786  3153.7696  985.5530   
43663                   276            4    419.4589    40.2681   12.5838   
43664                   280            1  24432.6088  2344.9921  732.8100   
...                     ...          ...         ...        ...       ...   
63358                  <NA>            7   1173.9600    93.9168   29.3490   
63359                  <NA>           10   1179.4700    94.3576   29.4868   
63360                  <NA>            7    548.9800    43.9184   13.7245   
63361                  <NA>            8   2384.0700   190.7256   59.6018   
63362                  <NA>           10   2419.0600   193.5248   60.4765   

                TotalDue  ...  \
SalesOrderID              ...   
43659         23153.2339  ...   
43660          1457.3288  ...   
43661         36865.8012  ...   
43663           472.3108  ...   
43664         27510.4109  ...   
...                  ...  ...   
63358          1297.2258  ...   
63359          1303.3144  ...   
63360           606.6229  ...   
63361          2634.3974  ...   
63362          2673.0613  ...   

             NUM_UNIQUE(sales_order_detail.YEAR(ModifiedDate))  \
SalesOrderID                                                     
43659                                                        1   
43660                                                        1   
43661                                                        1   
43663                                                        1   
43664                                                        1   
...                                                        ...   
63358                                                        1   
63359                                                        1   
63360                                                        1   
63361                                                        1   
63362                                                        1   

              NUM_UNIQUE(sales_order_detail.YEAR(StartDate))  \
SalesOrderID                                                   
43659                                                      1   
43660                                                      1   
43661                                                      1   
43663                                                      1   
43664                                                      1   
...                                                      ...   
63358                                                      1   
63359                                                      1   
63360                                                      1   
63361           

In [48]:
featuretools_train = getml.data.DataFrame.from_pandas(featuretools_train_pd, "featuretools_train")
featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test_pd, "featuretools_test")

In [49]:
featuretools_train.set_role("churn", getml.data.roles.target)
featuretools_train.set_role(featuretools_train.roles.unused_float, getml.data.roles.numerical)
featuretools_train.set_role(featuretools_train.roles.unused_string, getml.data.roles.categorical)

featuretools_train

name    churn   RevisionNumber   OnlineOrderFlag   SalesPersonID   ...   MEAN(store.SalesPersonID)   MIN(store.SalesPersonID)
 role   target   categorical      categorical       categorical     ...                   numerical                  numerical
    0        0   8                0                 279             ...                        279                        279 
    1        0   8                0                 279             ...                        279                        279 
    2        0   8                0                 282             ...                        282                        282 
    3        0   8                0                 276             ...                        276                        276 
    4        0   8                0                 280             ...                        280                        280 
           ...   ...              ...               ...                                         ...                        ...
15820        1   8                1                 NULL            ...                        nan                        nan 
15821        1   8                1                 NULL            ...                        nan                        nan 
15822        1   8                1                 NULL            ...                        nan                        nan 
15823        1   8                1                 NULL            ...                        nan                        nan 
15824        1   8                1                 NULL            ...                        nan                        nan 

 name   SKEW(store.SalesPersonID)   STD(store.SalesPersonID)   SUM(store.SalesPersonID)
 role                   numerical                  numerical                  numerical
    0                          0                          0                       22320
    1                          0                          0                       22320
    2                          0                          0                       20868
    3                          0                          0                       10764
    4                          0                          0                       10640
                              ...                        ...                        ...
15820                        nan                        nan                           0
15821                        nan                        nan                           0
15822                        nan                        nan                           0
15823                        nan                        nan                           0
15824                        nan                        nan                           0


15825 rows x 227 columns
memory usage: 22.03 MB
type: getml.DataFrame

In [50]:
featuretools_test.set_role("churn", getml.data.roles.target)
featuretools_test.set_role(featuretools_test.roles.unused_float, getml.data.roles.numerical)
featuretools_test.set_role(featuretools_test.roles.unused_string, getml.data.roles.categorical)

featuretools_test

name    churn   RevisionNumber   OnlineOrderFlag   SalesPersonID   ...   MEAN(store.SalesPersonID)   MIN(store.SalesPersonID)
role   target   categorical      categorical       categorical     ...                   numerical                  numerical
   0        0   8                0                 282             ...                        282                        282 
   1        0   8                0                 283             ...                        283                        283 
   2        0   8                0                 283             ...                        283                        283 
   3        0   8                0                 279             ...                        279                        279 
   4        0   8                0                 277             ...                        277                        277 
          ...   ...              ...               ...                                         ...                        ...
3874        1   8                1                 NULL            ...                        nan                        nan 
3875        1   8                1                 NULL            ...                        nan                        nan 
3876        1   8                1                 NULL            ...                        nan                        nan 
3877        1   8                1                 NULL            ...                        nan                        nan 
3878        1   8                1                 NULL            ...                        nan                        nan 

name   SKEW(store.SalesPersonID)   STD(store.SalesPersonID)   SUM(store.SalesPersonID)
role                   numerical                  numerical                  numerical
   0                          0                          0                       20868
   1                          0                          0                       10754
   2                          0                          0                       10754
   3                          0                          0                       22320
   4                          0                          0                       21052
                             ...                        ...                        ...
3874                        nan                        nan                           0
3875                        nan                        nan                           0
3876                        nan                        nan                           0
3877                        nan                        nan                           0
3878                        nan                        nan                           0


3879 rows x 227 columns
memory usage: 5.40 MB
type: getml.DataFrame

We train an untuned XGBoostRegressor on top of featuretools' features, just like we have done for getML's features.

Since some of featuretools features are categorical, we allow the pipeline to include these features as well. Other features contain NaN values, which is why we also apply getML's Imputation preprocessor.

In [51]:
imputation = getml.preprocessors.Imputation()

predictor = getml.predictors.XGBoostClassifier(n_jobs=1)

pipe3 = getml.Pipeline(
    tags=['featuretools'],
    preprocessors=[imputation],
    predictors=[predictor],
    include_categorical=True,
)

pipe3

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [52]:
pipe3.fit(featuretools_train)

Checking data model...

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Checking... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


The pipeline check generated 0 issues labeled INFO and 19 issues labeled WARNING.

To see the issues in full, run .check() on the pipeline.

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  XGBoost: Training as predictor... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:06


Trained pipeline.

Time taken: 0:00:07.203621.



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=True,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostClassifier'],
         preprocessors=['Imputation'],
         share_selected_features=0.5,
         tags=['featuretools'])

In [53]:
featuretools_score = pipe3.score(featuretools_test)
featuretools_score

  Staging... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00
  Preprocessing... ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 00:00


date time             set used             target   accuracy       auc   cross entropy
0   2024-09-12 13:04:10   featuretools_train   churn      0.9144    0.9697          0.2329
1   2024-09-12 13:04:10   featuretools_test    churn      0.9118    0.966           0.2428

#### 2.6 Features

The most important features look as follows:

In [54]:
pipe1.features.to_sql()[pipe1.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_301";

CREATE TABLE "FEATURE_1_301" AS
SELECT MIN( t1."orderdate" - t2."t4__startdate" ) AS "feature_1_301",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "SALES_ORDER_DETAIL__STAGING_TABLE_2" t2
ON t1."salesorderid" = t2."salesorderid"
GROUP BY t1.rowid;
```

In [55]:
pipe2.features.to_sql()[pipe2.features.sort(by="importances")[0].name]

```sql
DROP TABLE IF EXISTS "FEATURE_1_1";

CREATE TABLE "FEATURE_1_1" AS
SELECT AVG( 
    CASE
        WHEN ( t1."orderdate" - t2."t4__startdate" > 65976703.448276 ) AND ( t1."salespersonidcat" IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) ) AND ( t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" > 0.741158 ) THEN 1.944903076268523
        WHEN ( t1."orderdate" - t2."t4__startdate" > 65976703.448276 ) AND ( t1."salespersonidcat" IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) ) AND ( t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" <= 0.741158 OR t2."month__strftime__m__modifieddate__mapping_2_target_1_avg" IS NULL ) THEN -1.952864243920228
        WHEN ( t1."orderdate" - t2."t4__startdate" > 65976703.448276 ) AND ( t1."salespersonidcat" NOT IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) OR t1."salespersonidcat" IS NULL ) AND ( t1."territoryidcat" IN ( '6' ) ) THEN 0.3081692636953738
        WHEN ( t1."orderdate" - t2."t4__startdate" > 65976703.448276 ) AND ( t1."salespersonidcat" NOT IN ( '279', '282', '276', '280', '283', '277', '275', '278', '281', '289', '290', '287', '284', '286', '288', '285' ) OR t1."salespersonidcat" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '6' ) OR t1."territoryidcat" IS NULL ) THEN 1.657101846423358
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 65976703.448276 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" IN ( '2', '10', '9', '8' ) ) AND ( t2."t3__productmodelid" IN ( '1', '14', '31', '23', '19', '21', '12', '2', '10', '17', '22', '35', '34', '36', '24', '18', '20', '25', '28', '37', '26', '29', '30', '32', '16', '9', '6', '15', '13', '8', '7', '3', '5', '104', '106', '60', '61', '52', '54', '55', '56', '42', '45', '46', '50', '51', '123', '124', '125', '126', '77', '78', '115', '116', '38', '114', '118', '119', '107', '127', '128', '79', '80', '66', '65', '67', '62', '63', '68', '70', '53', '103', '47', '48', '102', '99', '101', '98', '95', '97' ) ) THEN -1.994292123763791
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 65976703.448276 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" IN ( '2', '10', '9', '8' ) ) AND ( t2."t3__productmodelid" NOT IN ( '1', '14', '31', '23', '19', '21', '12', '2', '10', '17', '22', '35', '34', '36', '24', '18', '20', '25', '28', '37', '26', '29', '30', '32', '16', '9', '6', '15', '13', '8', '7', '3', '5', '104', '106', '60', '61', '52', '54', '55', '56', '42', '45', '46', '50', '51', '123', '124', '125', '126', '77', '78', '115', '116', '38', '114', '118', '119', '107', '127', '128', '79', '80', '66', '65', '67', '62', '63', '68', '70', '53', '103', '47', '48', '102', '99', '101', '98', '95', '97' ) OR t2."t3__productmodelid" IS NULL ) THEN 2.393874664272962
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 65976703.448276 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '2', '10', '9', '8' ) OR t1."territoryidcat" IS NULL ) AND ( t2."t3__productmodelid" IN ( '1', '14', '31', '23', '19', '21', '2', '10', '11', '4', '17', '35', '33', '24', '18', '28', '37', '26', '29', '30', '32', '16', '9', '6', '13', '8', '7', '3', '5', '104', '106', '59', '60', '61', '52', '55', '42', '45', '46', '50', '51', '123', '124', '125', '78', '116', '38', '118', '119', '107', '127', '128', '79', '80', '81', '84', '66', '67', '62', '63', '64', '68', '69', '70', '53', '103', '47', '48', '102', '99', '100', '101', '98', '95', '97' ) ) THEN -1.575213171680517
        WHEN ( t1."orderdate" - t2."t4__startdate" <= 65976703.448276 OR t1."orderdate" IS NULL OR t2."t4__startdate" IS NULL ) AND ( t1."territoryidcat" NOT IN ( '2', '10', '9', '8' ) OR t1."territoryidcat" IS NULL ) AND ( t2."t3__productmodelid" NOT IN ( '1', '14', '31', '23', '19', '21', '2', '10', '11', '4', '17', '35', '33', '24', '18', '28', '37', '26', '29', '30', '32', '16', '9', '6', '13', '8', '7', '3', '5', '104', '106', '59', '60', '61', '52', '55', '42', '45', '46', '50', '51', '123', '124', '125', '78', '116', '38', '118', '119', '107', '127', '128', '79', '80', '81', '84', '66', '67', '62', '63', '64', '68', '69', '70', '53', '103', '47', '48', '102', '99', '100', '101', '98', '95', '97' ) OR t2."t3__productmodelid" IS NULL ) THEN -0.7654079549065077
        ELSE NULL
    END
) AS "feature_1_1",
       t1.rowid AS rownum
FROM "POPULATION__STAGING_TABLE_1" t1
INNER JOIN "SALES_ORDER_DETAIL__STAGING_TABLE_2" t2
ON t1."salesorderid" = t2."salesorderid"
GROUP BY t1.rowid;
```

#### 2.7 Productionization

It is possible to productionize the pipeline by transpiling the features into production-ready SQL code. Please also refer to getML's `sqlite3` and `spark` modules.


In [56]:
# Creates a folder containing the SQL code.
pipe2.features.to_sql().save("adventure_works", remove=True)

In [57]:
# Creates a folder containing the SQL code.
pipe2.features.to_sql(dialect=getml.pipeline.dialect.spark_sql).save("adventure_works_spark", remove=True)

#### 2.8 Discussion

For a more convenient overview, we summarize our results into a table.

In [58]:
scores = [fastprop_score, relboost_score, featuretools_score]
pd.DataFrame(
    data={
        'Name': ['getML: FastProp', 'getML: Relboost', 'featuretools'],
        'Accuracy': [f"{score.accuracy:.2%}" for score in scores],
        'AUC': [f"{score.auc:.4f}" for score in scores],
        'Cross entropy': [f"{score.cross_entropy:.4f}" for score in scores]
    }
)

Name Accuracy     AUC Cross entropy
0  getML: FastProp   91.42%  0.9723        0.2199
1  getML: Relboost   92.86%  0.9788        0.1888
2     featuretools   91.18%  0.9660        0.2428

The picture we get is very consistent: Relboost outperforms FastProp and FastProp outperforms featuretools for all three measures.

These results also clearly demonstrate that this is a synthetic dataset - in the real-world, customer churn can not be predicted at this level of accuracy.

In [ ]:
getml.engine.shutdown()

### 3. Conclusion

We demonstrated how getML can be used for a classic customer churn project. We have also benchmarked against featuretools. We found that getML outperforms featuretools.

## References

Motl, Jan, and Oliver Schulte. "The CTU prague relational learning repository." arXiv preprint arXiv:1511.03086 (2015).